# Exploratory Data Analysis for METR-LA, PEMS-BAY and Incheon

## Loading Data

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import geopandas as gpd

In [ ]:
df_la = pd.read_hdf("../datasets/METRLA/metr-la.h5")
df_la.head().reset_index().iloc[:, :9]

In [ ]:
df_pems = pd.read_hdf("../datasets/PEMSBAY/pems-bay.h5")
df_pems.head().reset_index().iloc[:, :9]

In [ ]:
df_imc: pd.DataFrame = pd.read_hdf("../datasets/metr-imc/metr-imc.h5")
df_imc.head().iloc[:, :9]

In [ ]:
print("Is time series index:", isinstance(df_imc.index, pd.DatetimeIndex))
df_imc_time_diff = df_imc.index.to_series().diff().dropna()
print("Is data hourly:", (df_imc_time_diff == pd.Timedelta(hours=1)).all())

In [ ]:
# 인덱스를 시간 차이로 확인
time_diffs = df_imc.index.to_series().diff()

# 1시간 간격이 아닌 부분 찾기
gaps = time_diffs[time_diffs != pd.Timedelta(hours=1)]

# 결과 출력
if gaps.empty:
    print("인덱스는 1시간 간격으로 되어 있습니다.")
else:
    print("인덱스가 1시간 간격이 아닌 부분:")
    for idx in gaps.index:
        print(f"시작 지점: {df_imc.index[df_imc.index.get_loc(idx) - 1]}, 끝 지점: {idx}")

In [ ]:
df_imc_sorted = df_imc.sort_index()

# 인덱스를 시간 차이로 확인
time_diffs = df_imc_sorted.index.to_series().diff()

# 1시간 간격이 아닌 부분 찾기
gaps = time_diffs[time_diffs != pd.Timedelta(hours=1)]

# 결과 출력
if gaps.empty:
    print("인덱스는 1시간 간격으로 되어 있습니다.")
else:
    print("인덱스가 1시간 간격이 아닌 부분:")
    for idx in gaps.index:
        print(f"시작 지점: {df_imc_sorted.index[df_imc_sorted.index.get_loc(idx) - 1]}, 끝 지점: {idx}")

### Column and Data info

In [ ]:
print("-- METR-LA\n", f"dtype: {df_la.dtypes.unique().tolist()}\n", f"length: {len(df_la.columns)}", end="\n\n")
print("-- PEMS-BAY\n", f"dtype: {df_pems.dtypes.unique().tolist()}\n", f"length: {len(df_pems.columns)}", end="\n\n")
print("-- Incheon\n", f"dtype: {df_la.dtypes.unique().tolist()}\n", f"length: {len(df_imc.columns)}")

### Index info

In [ ]:
print("-- METR-LA\n", f"dtype: {df_la.index.dtype}\n", f"length: {len(df_la.index)}", end="\n\n")
print("-- PEMS-BAY\n", f"dtype: {df_pems.index.dtype}\n", f"length: {len(df_pems.index)}", end="\n\n")
print("-- Incheon\n", f"dtype: {df_imc.index.dtype}\n", f"length: {len(df_imc.index)}")

### Missing Data (결측값)

In [ ]:
print("-- METR-LA")
print(f"   No Missing: {len(df_la.columns[df_la.isnull().sum() == 0])}")
print(f"   <=100: {len(df_la.columns[df_la.isnull().sum() <= 100])}")
print(f"   >100: {len(df_la.columns[df_la.isnull().sum() > 100])}", end="\n\n")

print("-- PEMS-BAY")
print(f"   No Missing: {len(df_pems.columns[df_pems.isnull().sum() == 0])}")
print(f"   <=100: {len(df_pems.columns[df_pems.isnull().sum() <= 100])}")
print(f"   >100: {len(df_pems.columns[df_pems.isnull().sum() > 100])}", end="\n\n")

print("-- Incheon")
print(f"   No Missing: {len(df_imc.columns[df_imc.isnull().sum() == 0])}")
print(f"   <=100: {len(df_imc.columns[df_imc.isnull().sum() <= 100])}")
print(f"   >100: {len(df_imc.columns[df_imc.isnull().sum() > 100])}")
print(f"   <500: {len(df_imc.columns[df_imc.isnull().sum() < 500])}")
print(f"   <750: {len(df_imc.columns[df_imc.isnull().sum() < 750])}")

#### Visualization of Missing Data

In [ ]:
import missingno as msno

def visualize_missingno(df: pd.DataFrame):
    msno.matrix(df)
    plt.show()

    msno.heatmap(df, labels=False)
    plt.show()

    msno.bar(df)
    plt.show()

missingno는 결측값 시각화 패키지이며 각 함수는 다음의 의미와 같다.
1. matrix: 빈 공간은 결측값을 나타내며 이를 통해 결측 패턴을 파악.
2. heatmap: 열 간 결측값 상관 계수를 히트맵으로 표시. 변수 간 결측값의 연관성 파악.
3. bar: 결측값 갯수를 막대로 표시. 각 열(변수)의 결측값 갯수 파악.

In [ ]:
sample_rate = 0.1

data_columns = df_imc_sorted.columns
num_columns_to_select = int(len(data_columns) * sample_rate)
selected_columns = np.random.choice(data_columns, num_columns_to_select, replace=False)
sampled_df = df_imc_sorted[selected_columns]

print(f"샘플링된 데이터 프레임의 크기: {sampled_df.shape}")
visualize_missingno(sampled_df)

Matrix에서 특정 날짜에서 결측값이 많아지는 경향을 볼 수 있다. 특히 데이터 시작 부분의 일정 기간 동안 결측값이 많이 관찰되었다. 특정 날짜에서 대부분의 센서들이 동작하지 않았을 가능성이 높다. 또한, 데이터 시작 부분의 데이터는 제외하는 것을 고려할 수 있다.

Heatmap 분석에서는 결측값의 상관관계가 높은 경우가 많이 확인되었다. 일반적으로 이 경우 결측값 형태가 MAR(Missing At Random)이라고 판단한다. 다만 각 열이 모두 독립적인 센서임을 감안할 때, 각 센서가 영향을 주었을 가능성은 낮고 중앙 시스템에서 문제가 됬을 가능성이 높기 때문에 이 점을 감안할 필요가 있다.

Bar 분석에서는 대부분 결측치가 많다는 것을 확인할 수 있다.

In [ ]:
sample_traffic_data = df_imc[df_imc.columns[df_imc.isnull().sum() == 0][0]]

plt.figure(figsize=(12, 2))
sns.lineplot(sample_traffic_data)
plt.title(f"{sample_traffic_data.name} Road Traffic Data")
plt.xlabel("Date Time")
plt.show()

최종적으로는 결측값을 포함한 전체 데이터셋을 기반으로 모델을 생성하고 성능을 평가해야 하지만 결측값 문제는 현재 집중하고 있는 연구 방향과는 독립적인 문제로 판단된다. 또한, 모든 데이터를 포함하면 데이터의 크기가 커져 학습 시간이 길어지는 문제도 있다. 따라서 현재는 데이터를 줄여 효율성을 높이고자 먼저 결측값이 없는 데이터를 사용하여 모델을 생성하고, 이 모델이 유효한지 판단한 후, 결측값 문제를 추가로 해결하는 방향으로 연구를 진행하고자 한다.

In [ ]:
from songdo_traffic_core.dataset.gis.view_layer.metr import SensorView

no_missing_columns = df_imc.columns[df_imc.isnull().sum() == 0].to_list()
less_500_missing_columns = df_imc.columns[df_imc.isnull().sum() < 500].to_list()
less_750_missing_columns = df_imc.columns[df_imc.isnull().sum() < 750].to_list()

In [ ]:
view = SensorView("../datasets/metr-imc/graph_sensor_locations.csv")
view.set_filter(no_missing_columns)
view.export_to_file("../datasets/metr-imc/miscellaneous", "no_missing.shp")
view.set_filter(less_500_missing_columns)
view.export_to_file("../datasets/metr-imc/miscellaneous", "missing_500.shp")
view.set_filter(less_750_missing_columns)
view.export_to_file("../datasets/metr-imc/miscellaneous", "missing_750.shp")

결측치가 없는 데이터를 사용하는 것이 가장 이상적이지만, 이 경우 노드의 수가 너무 줄어들어 공간 정보가 모델에 충분히 반영되지 않을 가능성이 있다. 따라서, 기존에 많이 사용하는 METR-LA, PEMS-BAY 등의 데이터셋 크기와 유사한 수준으로 조정하였다. 결측치 허용 기준을 여러 개 비교한 결과, 500 정도가 기존 데이터셋과 유사한 크기를 유지하면서도 공간 정보를 적절히 반영할 수 있을 것으로 예상되었다.

- 결측치가 하나도 없는 센서 노드
![Missing_0](../docs/Missing_0.png)

- 결측치 500개 미만의 센서 노드
![Missing_500](../docs/Missing_500.png)

- 결측치 750개 미미나의 센서 노드
![Missing_750](../docs/Missing_750.png)

이에 따라 500개 미만의 결측치를 가진 센서 노드만 별도로 추출하여 데이터셋을 다시 생성하였다.

In [ ]:
from songdo_traffic_core.dataset.metr_imc.generator import MetrImcSubsetGenerator

generator = MetrImcSubsetGenerator(
    nodelink_dir="../datasets/metr-imc/nodelink",
    imcrts_dir="../datasets/metr-imc/imcrts",
    metr_imc_dir="../datasets/metr-imc/",
)

In [ ]:
l500_missing_df, l500_columns = generator.process_metr_imc(less_500_missing_columns)
l500_missing_df.info()

In [ ]:
visualize_missingno(l500_missing_df)

위 그래프의 경우 원래데이터와 다르게 결측치 분포의 편차는 많이 제거되었다. 특정 기간동안 보이던 결측치 또한 많이 제거되어 있음을 확인할 수 있다. 센서마다 보이던 결측치의 상관관계도 많이 줄어들었다. 그러나 여전히 상관관계가 있는 결측치들이 있으며 이것은 여전히 결측치가 MAR임을 알 수 있다.

In [ ]:
from sklearn.cluster import KMeans


missing_patterns = l500_missing_df.isnull().astype(int)
kmeans = KMeans(n_clusters=8, random_state=42)
clusters = kmeans.fit_predict(missing_patterns.T)
clustered_df = pd.DataFrame({'sensor': missing_patterns.columns, 'cluster': clusters})
for cluster in set(clusters):
    print(f"Cluster {cluster}: {clustered_df[clustered_df['cluster'] == cluster]['sensor'].tolist()}")

In [ ]:
visualize_clusters = False

if visualize_clusters:
    for cluster in set(clusters):
        cluster_sensors = clustered_df[clustered_df['cluster'] == cluster]['sensor'].tolist()
        plt.figure(figsize=(6, 2))
        msno.matrix(l500_missing_df[cluster_sensors])
        plt.title(f'Matrix Plot for Cluster {cluster}')
        plt.show()

        plt.figure(figsize=(6, 2))
        msno.heatmap(l500_missing_df[cluster_sensors], labels=False)
        plt.title(f'Heatmap Plot for Cluster {cluster}')
        plt.show()

In [ ]:
from songdo_traffic_core.dataset.interpolator import IterativeRandomForestInterpolator


interpolator = IterativeRandomForestInterpolator()

In [ ]:
df_int, df_int_columns = generator.process_metr_imc(
    less_500_missing_columns, interpolate_filter=interpolator
)
df_int.info()

In [ ]:
visualize_missingno(df_int)